# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
weather_data = pd.read_csv('output_data/cities.csv')
weather_data = weather_data.drop(columns= "Unnamed: 0")
weather_data

,city,lat,lng,max temp,humidity,wind speed,cloudiness,country,date
0,Jamestown,42.0970,-79.2353,76.59,43,9.22,0,US,1666723694
1,San Vicente de Cañete,-13.0751,-76.3835,67.21,70,9.44,13,PE,1666724154
2,Altay,47.8667,88.1167,36.21,30,4.63,100,CN,1666723984
3,Barranco,9.5167,-82.7000,87.91,62,6.06,88,PA,1666724155
4,Ushuaia,-54.8000,-68.3000,47.82,49,6.91,0,AR,1666724155
...,...,...,...,...,...,...,...,...,...
550,Grasse,43.6667,6.9167,63.09,91,3.44,0,FR,1666724354
551,Vysokovsk,56.3167,36.5500,31.24,76,5.32,48,RU,1666724354
552,Banda Aceh,5.5577,95.3222,79.39,82,9.31,86,ID,1666724355
553,Flagstaff,35.1981,-111.6513,58.82,27,3.44,0,US,1666724089


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
gmaps.configure(api_key=g_key)

In [56]:
locations = weather_data[['lat', 'lng']]
humidity = weather_data["humidity"]
max_humidity = float(humidity.max())

layout={'width': '600px',
        'height': '420px',
        'padding': '3px',
        'border': '1px solid black',
        'margin': '0 auto 0 auto'}

fig = gmaps.figure(layout=layout, center = [0, 0], zoom_level=1.2)
humid_heatmap = gmaps.heatmap_layer(locations, weights=humidity)
fig.add_layer(heatmap)
fig


Figure(layout=FigureLayout(border='1px solid black', height='420px', margin='0 auto 0 auto', padding='3px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
ideal_df = weather_data.loc[(weather_data["max temp"] > 75) &
                                         (weather_data["max temp"] < 95) &
                                         (weather_data["wind speed"] < 10) &
                                         (weather_data["cloudiness"] <= 25)]
                                    
ideal_df

,city,lat,lng,max temp,humidity,wind speed,cloudiness,country,date
0,Jamestown,42.0970,-79.2353,76.59,43,9.22,0,US,1666723694
10,Weston,26.1004,-80.3998,87.01,50,7.00,0,US,1666724086
43,Fartura,-23.3883,-49.5100,81.88,43,6.55,0,BR,1666724167
54,Kāvali,14.9167,79.9833,75.36,63,7.40,1,IN,1666724171
87,Castro,-24.7911,-50.0119,75.43,50,5.66,0,BR,1666723940
128,George Town,5.4112,100.3354,78.73,93,2.30,20,MY,1666723732
135,Madera,37.1666,-119.8343,75.87,25,3.00,3,US,1666724162
136,Guerrero Negro,27.9769,-114.0611,75.97,31,3.11,0,MX,1666724196
149,Dwārka,22.2394,68.9678,80.19,63,8.77,0,IN,1666723687
155,Dholka,22.7167,72.4667,79.18,42,5.59,0,IN,1666724201


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [43]:
ideal_df = ideal_df.reset_index(drop=True)
ideal_df = ideal_df.drop(['hotel name'], axis=1)
ideal_df

,city,lat,lng,max temp,humidity,wind speed,cloudiness,country,date,hotel hame
0,Jamestown,42.0970,-79.2353,76.59,43,9.22,0,US,1666723694,Chautauqua Harbor Hotel
1,Weston,26.1004,-80.3998,87.01,50,7.00,0,US,1666724086,Courtyard by Marriott Fort Lauderdale Weston
2,Fartura,-23.3883,-49.5100,81.88,43,6.55,0,BR,1666724167,Hotel Fartura Plaza
3,Kāvali,14.9167,79.9833,75.36,63,7.40,1,IN,1666724171,Hotel Mourya Paradise
4,Castro,-24.7911,-50.0119,75.43,50,5.66,0,BR,1666723940,Hotel Buganville Express
5,George Town,5.4112,100.3354,78.73,93,2.30,20,MY,1666723732,"Eastern & Oriental Hotel, Penang"
6,Madera,37.1666,-119.8343,75.87,25,3.00,3,US,1666724162,NaN
7,Guerrero Negro,27.9769,-114.0611,75.97,31,3.11,0,MX,1666724196,The Halfway Inn
8,Dwārka,22.2394,68.9678,80.19,63,8.77,0,IN,1666723687,"VITS Devbhumi, Dwarka"
9,Dholka,22.7167,72.4667,79.18,42,5.59,0,IN,1666724201,Royal Hotel


In [39]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in ideal_df.iterrows():
    
    target_coord = f"{row['lat']}, {row['lng']}"
    target_search = "hotel"
    target_radius = 5000
    params = {
        "location": target_coord,
        "keyword": target_search,
        "radius": target_radius,
        "key": g_key
        }
    try:
        response = requests.get(base_url, params=params).json()
        ideal_df.loc[index,'Hotel Name'] = response['results'][0]['name']
    except IndexError:
        print(f"hotel not found for {row['city']}..skipping")


hotel not found for Madera..skipping
hotel not found for Kirakira..skipping
hotel not found for Sompeta..skipping


In [52]:
hotel_df = ideal_df
hotel_df = hotel_df.rename(columns={'hotel hame': "Hotel Name",
                                    'city':'City',
                                    'country':'Country', 
                                    'lat':'Lat', 
                                    'lng':'Lng'})

hotel_df

,City,Lat,Lng,max temp,humidity,wind speed,cloudiness,Country,date,Hotel Name
0,Jamestown,42.0970,-79.2353,76.59,43,9.22,0,US,1666723694,Chautauqua Harbor Hotel
1,Weston,26.1004,-80.3998,87.01,50,7.00,0,US,1666724086,Courtyard by Marriott Fort Lauderdale Weston
2,Fartura,-23.3883,-49.5100,81.88,43,6.55,0,BR,1666724167,Hotel Fartura Plaza
3,Kāvali,14.9167,79.9833,75.36,63,7.40,1,IN,1666724171,Hotel Mourya Paradise
4,Castro,-24.7911,-50.0119,75.43,50,5.66,0,BR,1666723940,Hotel Buganville Express
5,George Town,5.4112,100.3354,78.73,93,2.30,20,MY,1666723732,"Eastern & Oriental Hotel, Penang"
6,Madera,37.1666,-119.8343,75.87,25,3.00,3,US,1666724162,NaN
7,Guerrero Negro,27.9769,-114.0611,75.97,31,3.11,0,MX,1666724196,The Halfway Inn
8,Dwārka,22.2394,68.9678,80.19,63,8.77,0,IN,1666723687,"VITS Devbhumi, Dwarka"
9,Dholka,22.7167,72.4667,79.18,42,5.59,0,IN,1666724201,Royal Hotel


In [53]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [54]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='420px', margin='0 auto 0 auto', padding='3px', wi…